#### For Colab

In [2]:
"""
function ClickConnect(){
    console.log("Working");
    document.querySelector("colab-toolbar-button").click() 
}
var i = setInterval(ClickConnect, 900000)
clearInterval(i)
"""

'\nfunction ClickConnect(){\n    console.log("Working");\n    document.querySelector("colab-toolbar-button").click() \n}\nvar i = setInterval(ClickConnect, 900000)\nclearInterval(i)\n'

In [3]:
!nvidia-smi

Wed Mar 17 15:09:01 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.56       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P0    30W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [5]:
drive_dir = '/content/drive/My Drive/MAGMA: Summarization/'

#### Install Libraries

In [6]:
!cd '/content/drive/My Drive/MAGMA: Summarization/transformers/'; pip install -q -e .
!pip install -q -U sentence-transformers
!pip install -q -U wandb
!pip install -q -U rouge-score sacrebleu

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 3.2MB 4.1MB/s 
     |████████████████████████████████| 890kB 26.9MB/s 
     |████████████████████████████████| 71kB 3.5MB/s 
     |████████████████████████████████| 1.2MB 7.4MB/s 
     |████████████████████████████████| 2.0MB 4.3MB/s 
     |████████████████████████████████| 102kB 3.1MB/s 
     |████████████████████████████████| 133kB 40.2MB/s 
     |████████████████████████████████| 163kB 39.1MB/s 
     |████████████████████████████████| 71kB 8.2MB/s 
     |████████████████████████████████| 61kB 3.2MB/s 


In [7]:
print("Setting up colab environment")
!pip uninstall -y -q pyarrow
!pip install -q -U ray[tune]
!pip install -q ray[debug]

Setting up colab environment
     |████████████████████████████████| 47.5MB 65kB/s 
     |████████████████████████████████| 81kB 8.8MB/s 
     |████████████████████████████████| 3.2MB 37.8MB/s 
     |████████████████████████████████| 133kB 43.3MB/s 
     |████████████████████████████████| 81kB 8.5MB/s 
     |████████████████████████████████| 71kB 7.2MB/s 
     |████████████████████████████████| 204kB 41.0MB/s 
     |████████████████████████████████| 1.3MB 35.7MB/s 
     |████████████████████████████████| 317kB 39.0MB/s 
     |████████████████████████████████| 71kB 7.7MB/s 
     |████████████████████████████████| 296kB 40.1MB/s 
     |████████████████████████████████| 143kB 43.3MB/s 


In [8]:
import sys

print("\n##### sys.path original #####")
for p in sys.path:
    print(f"'{p}'")
print()

import site
site.main()

print("\n##### sys.path after site.main() #####")
for p in sys.path:
    print(f"'{p}'")
print()


##### sys.path original #####
''
'/content'
'/env/python'
'/usr/lib/python37.zip'
'/usr/lib/python3.7'
'/usr/lib/python3.7/lib-dynload'
'/usr/local/lib/python3.7/dist-packages'
'/usr/lib/python3/dist-packages'
'/usr/local/lib/python3.7/dist-packages/IPython/extensions'
'/root/.ipython'


##### sys.path after site.main() #####
'/content'
'/env/python'
'/usr/lib/python37.zip'
'/usr/lib/python3.7'
'/usr/lib/python3.7/lib-dynload'
'/usr/local/lib/python3.7/dist-packages'
'/usr/lib/python3/dist-packages'
'/usr/local/lib/python3.7/dist-packages/IPython/extensions'
'/root/.ipython'
'/content/drive/My Drive/MAGMA: Summarization/transformers/src'



### **Config**

In [9]:
import sys
sys.path.insert(0, drive_dir)
import config

import wandb
wandb.login()

project_name = 'ft_pegasus_bull_para_embed_merged_overlaps_bybook'
%env WANDB_PROJECT=$project_name

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


env: WANDB_PROJECT=ft_pegasus_bybook_para_wordembed_merge


In [10]:
finetune_script = '"'+drive_dir+'transformers/examples/seq2seq/finetune_trainer.py"'
eval_script = '"'+drive_dir+'transformers/examples/seq2seq/run_eval.py"'
gps_script = '"'+drive_dir+'transformers/examples/seq2seq/run_eval_search.py"'

In [11]:
model_name_or_path = 'google/pegasus-large'

In [12]:
data_dir = '"'+drive_dir+'datasets/bullet_paragraph_embeddings/pegasus/st/merged_overlaps/bybook/"'

source_val_dir = data_dir[:-1] + 'val.source"'
reference_val_dir = data_dir[:-1] + 'val.target"'

source_test_dir = data_dir[:-1] + 'test.source"'
reference_test_dir = data_dir[:-1] + 'test.target"'

In [13]:
GAS = 64
LR = 5e-5

output_dir = '"'+drive_dir+'fine-tuning/'+project_name+'_gas'+\
    str(GAS)+'_lr' + str(LR) + '"'

### Evaluation Out of the Box

#### Validation

In [ ]:
eval_model_name_or_path = '"google/pegasus-large"'

save_dir = output_dir[:-1] + '/'+eval_model_name_or_path[1:-1].replace('/', '?')+\
    '_bull_para_embed_merged_overlaps_bybook_val.txt"'
score_dir = output_dir[:-1] + '/'+eval_model_name_or_path[1:-1].replace('/', '?')+\
    '_bull_para_embed_merged_overlaps_bybook_val.json"'

!python3 $eval_script \
$eval_model_name_or_path \
$source_val_dir \
$save_dir \
--reference_path $reference_val_dir \
--score_path $score_dir \
--bs 8 \
--dump-args \
--min_length $config.ONE_BULLET_MIN_LEN \
--max_length $config.ONE_BULLET_MAX_LEN

100% 306M/306M [00:13<00:00, 22.3MB/s]
parsed the following generate kwargs: {'min_length': 10, 'max_length': 150}
Downloading: 100% 2.87k/2.87k [00:00<00:00, 4.27MB/s]
Downloading: 100% 2.28G/2.28G [00:51<00:00, 44.5MB/s]
Downloading: 100% 1.91M/1.91M [00:00<00:00, 43.8MB/s]
Downloading: 100% 65.0/65.0 [00:00<00:00, 115kB/s]
Downloading: 100% 88.0/88.0 [00:00<00:00, 157kB/s]
  0% 0/19 [00:00<?, ?it/s]2021-03-15 11:05:34.282325: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
100% 19/19 [02:46<00:00,  8.78s/it]
{'rouge1_precision': 28.360000000000003, 'rouge1_recall': 30.049999999999997, 'rouge1_fmeasure': 26.919999999999998, 'rouge2_precision': 11.31, 'rouge2_recall': 11.09, 'rouge2_fmeasure': 10.25, 'rougeL_precision': 21.94, 'rougeL_recall': 23.29, 'rougeL_fmeasure': 20.849999999999998, 'rougeLsum_precision': 23.73, 'rougeLsum_recall': 24.46, 'rougeLsum_fmeasure': 22.2, 'sentence_distilroberta_cosine': 56.34734034

#### Test

In [ ]:
eval_model_name_or_path = '"google/pegasus-large"'

save_dir = output_dir[:-1] + '/'+eval_model_name_or_path[1:-1].replace('/', '?')+\
    '_bull_para_embed_merged_overlaps_bybook_test.txt"'
score_dir = output_dir[:-1] + '/'+eval_model_name_or_path[1:-1].replace('/', '?')+\
    '_bull_para_embed_merged_overlaps_bybook_test.json"'

!python3 $eval_script \
$eval_model_name_or_path \
$source_test_dir \
$save_dir \
--reference_path $reference_test_dir \
--score_path $score_dir \
--bs 8 \
--dump-args \
--min_length $config.ONE_BULLET_MIN_LEN \
--max_length $config.ONE_BULLET_MAX_LEN

parsed the following generate kwargs: {'min_length': 10, 'max_length': 150}
  0% 0/28 [00:00<?, ?it/s]2021-03-15 11:09:58.968418: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
100% 28/28 [04:21<00:00,  9.32s/it]
{'rouge1_precision': 27.810000000000002, 'rouge1_recall': 32.379999999999995, 'rouge1_fmeasure': 28.04, 'rouge2_precision': 9.6, 'rouge2_recall': 10.91, 'rouge2_fmeasure': 9.48, 'rougeL_precision': 20.48, 'rougeL_recall': 23.94, 'rougeL_fmeasure': 20.669999999999998, 'rougeLsum_precision': 22.2, 'rougeLsum_recall': 25.56, 'rougeLsum_fmeasure': 22.25, 'sentence_distilroberta_cosine': 56.46427273750305, 'w2v_cosine': 99.99693632125854, 'n_obs': 222, 'runtime': 261, 'seconds_per_sample': 1.1757, 'min_length': 10, 'max_length': 150}


### Fine-tuning

In [ ]:
from transformers import AutoConfig
model_config = AutoConfig.from_pretrained(model_name_or_path)
model_config.min_length = config.ONE_BULLET_MIN_LEN
model_config.max_length = config.ONE_BULLET_MAX_LEN
model_config.num_beams = 2

model_config_dir = '"'+drive_dir+'fine-tuning/config/'+\
    model_name_or_path.replace('/', '?')+'_config"'
model_config.save_pretrained(model_config_dir[1:-1])

In [ ]:
!python3 $finetune_script \
--model_name_or_path $model_name_or_path \
--config_name $model_config_dir \
--tokenizer_name $model_name_or_path \
--data_dir $data_dir \
--freeze_embeds \
--adafactor \
--do_train \
--learning_rate $LR \
--label_smoothing_factor 0.1 \
--warmup_steps 0 \
--sortish_sampler \
--task summarization \
--max_source_length 1024 \
--max_target_length $config.ONE_BULLET_MAX_LEN \
--val_max_target_length $config.ONE_BULLET_MAX_LEN \
--max_steps 700 \
--logging_steps 7 --logging_first_step \
--per_device_train_batch_size 2 --per_device_eval_batch_size 8 \
--gradient_accumulation_steps $GAS \
--evaluation_strategy steps --eval_steps 7 --eval_beams 2 \
--predict_with_generate \
--save_steps 49 \
--output_dir $output_dir \
--overwrite_output_dir \
--seed $config.SEED \
--run_name $output_dir

Streaming output truncated to the last 5000 lines.
Batches: 100%|██████████| 1/1 [00:00<00:00, 175.83it/s]


Batches: 100%|██████████| 1/1 [00:00<00:00, 185.24it/s]


Batches: 100%|██████████| 1/1 [00:00<00:00, 182.48it/s]


Batches: 100%|██████████| 1/1 [00:00<00:00, 147.00it/s]


Batches: 100%|██████████| 1/1 [00:00<00:00, 147.47it/s]


Batches: 100%|██████████| 1/1 [00:00<00:00, 149.01it/s]


Batches: 100%|██████████| 1/1 [00:00<00:00, 139.65it/s]


Batches: 100%|██████████| 1/1 [00:00<00:00, 174.73it/s]


Batches: 100%|██████████| 1/1 [00:00<00:00, 154.60it/s]


Batches: 100%|██████████| 1/1 [00:00<00:00, 182.58it/s]


Batches: 100%|██████████| 1/1 [00:00<00:00, 179.87it/s]


Batches: 100%|██████████| 1/1 [00:00<00:00, 187.07it/s]


Batches: 100%|██████████| 1/1 [00:00<00:00, 168.72it/s]


Batches: 100%|██████████| 1/1 [00:00<00:00, 186.32it/s]


Batches: 100%|██████████| 1/1 [00:00<00:00, 179.97it/s]


Batches: 100%|██████████| 1/1 [00:00<00:00, 167.93it/s]


Batches: 100%|███████

### Evaluation before GPS

In [13]:
eval_model_name_or_path = output_dir[:-1] + '/checkpoint-539"'
print(eval_model_name_or_path+'\n')

save_dir = output_dir[:-1] + '/'+eval_model_name_or_path[1:-1].replace('/', '?')+'_bull_para_embed_merged_overlaps_bybook_val.txt"'
score_dir = output_dir[:-1] + '/'+eval_model_name_or_path[1:-1].replace('/', '?')+'_bull_para_embed_merged_overlaps_bybook_val.json"'

!python3 $eval_script \
$eval_model_name_or_path \
$source_val_dir \
$save_dir \
--reference_path $reference_val_dir \
--score_path $score_dir \
--bs 8 \
--dump-args \
--min_length $config.ONE_BULLET_MIN_LEN \
--max_length $config.ONE_BULLET_MAX_LEN

"/content/drive/My Drive/MAGMA: Summarization/fine-tuning/ft_pegasus_para_wordembed_merge_bybook_gas64_lr5e-05/checkpoint-539"

100% 306M/306M [00:34<00:00, 8.82MB/s]
parsed the following generate kwargs: {'min_length': 10, 'max_length': 150}
  0% 0/19 [00:00<?, ?it/s]2021-03-19 11:06:54.846555: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
100% 19/19 [00:34<00:00,  1.80s/it]
{'rouge1_precision': 38.35, 'rouge1_recall': 35.370000000000005, 'rouge1_fmeasure': 33.83, 'rouge2_precision': 18.08, 'rouge2_recall': 16.25, 'rouge2_fmeasure': 15.659999999999998, 'rougeL_precision': 30.869999999999997, 'rougeL_recall': 28.71, 'rougeL_fmeasure': 27.36, 'rougeLsum_precision': 32.6, 'rougeLsum_recall': 30.159999999999997, 'rougeLsum_fmeasure': 28.73, 'sentence_distilroberta_cosine': 62.69147992134094, 'w2v_cosine': 99.99613165855408, 'n_obs': 149, 'runtime': 34, 'seconds_per_sample': 0.2282, 'min_length': 10, 'max_length': 150}

In [14]:
eval_model_name_or_path = output_dir[:-1] + '/checkpoint-539"'
print(eval_model_name_or_path+'\n')

save_dir = output_dir[:-1] + '/'+eval_model_name_or_path[1:-1].replace('/', '?')+'_bull_para_embed_merged_overlaps_bybook_test.txt"'
score_dir = output_dir[:-1] + '/'+eval_model_name_or_path[1:-1].replace('/', '?')+'_bull_para_embed_merged_overlaps_bybook_test.json"'

!python3 $eval_script \
$eval_model_name_or_path \
$source_test_dir \
$save_dir \
--reference_path $reference_test_dir \
--score_path $score_dir \
--bs 8 \
--dump-args \
--min_length $config.ONE_BULLET_MIN_LEN \
--max_length $config.ONE_BULLET_MAX_LEN

"/content/drive/My Drive/MAGMA: Summarization/fine-tuning/ft_pegasus_para_wordembed_merge_bybook_gas64_lr5e-05/checkpoint-539"

parsed the following generate kwargs: {'min_length': 10, 'max_length': 150}
  0% 0/28 [00:00<?, ?it/s]2021-03-19 11:09:32.435733: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
100% 28/28 [00:47<00:00,  1.68s/it]
{'rouge1_precision': 36.84, 'rouge1_recall': 32.4, 'rouge1_fmeasure': 32.48, 'rouge2_precision': 15.98, 'rouge2_recall': 13.25, 'rouge2_fmeasure': 13.66, 'rougeL_precision': 28.96, 'rougeL_recall': 25.34, 'rougeL_fmeasure': 25.44, 'rougeLsum_precision': 30.669999999999998, 'rougeLsum_recall': 26.58, 'rougeLsum_fmeasure': 26.82, 'sentence_distilroberta_cosine': 60.23419499397278, 'w2v_cosine': 99.99663829803467, 'n_obs': 222, 'runtime': 47, 'seconds_per_sample': 0.2117, 'min_length': 10, 'max_length': 150}


### GPS

In [ ]:
eval_model_name_or_path = output_dir[:-1] + '/checkpoint-539"'
print(eval_model_name_or_path+'\n')

save_dir = output_dir[:-1] + '/'+eval_model_name_or_path[1:-1].replace('/', '?')+'_gps.txt"'
score_dir = output_dir[:-1] + '/'+eval_model_name_or_path[1:-1].replace('/', '?')+'_gps.json"'

!python3 $gps_script \
$eval_model_name_or_path \
$source_val_dir \
$save_dir \
--reference_path $reference_val_dir \
--score_path $score_dir \
--bs 8 \
--task summarization \
--search="min_length=5:10:15 length_penalty=0.1:1:1.9 num_beams=4:6 no_repeat_ngram_size=3:4:5" \
--dump-args \
--max_length $config.ONE_BULLET_MAX_LEN

"/content/drive/My Drive/MAGMA: Summarization/fine-tuning/ft_pegasus_para_wordembed_merge_bybook_gas64_lr5e-05/checkpoint-539"

  0% 0/19 [00:00<?, ?it/s]2021-03-15 11:15:44.895597: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
100% 19/19 [00:56<00:00,  3.00s/it]
100% 19/19 [00:57<00:00,  3.01s/it]
100% 19/19 [00:57<00:00,  3.04s/it]
100% 19/19 [01:21<00:00,  4.27s/it]
100% 19/19 [01:20<00:00,  4.22s/it]
100% 19/19 [01:24<00:00,  4.43s/it]
100% 19/19 [01:09<00:00,  3.68s/it]
100% 19/19 [01:15<00:00,  3.95s/it]
100% 19/19 [01:15<00:00,  3.96s/it]
100% 19/19 [01:56<00:00,  6.13s/it]
100% 19/19 [02:05<00:00,  6.62s/it]
100% 19/19 [02:09<00:00,  6.81s/it]
100% 19/19 [01:27<00:00,  4.61s/it]
100% 19/19 [01:33<00:00,  4.94s/it]
100% 19/19 [01:38<00:00,  5.17s/it]
100% 19/19 [02:25<00:00,  7.67s/it]
100% 19/19 [02:31<00:00,  7.99s/it]
100% 19/19 [02:35<00:00,  8.17s/it]
100% 19/19 [00:55<00:00,  2.91s/it]
100% 19/19 [00:5

In [ ]:
print('hello')

hello


### Evaluation

In [18]:
### Parameters found through Generative Parameter Search and visual representation
### check the file gps_pegasus_para_wordembed_merge

min_length = 10
length_penalty = 1
num_beams = 6
no_repeat_ngram_size = 3

In [21]:
eval_model_name_or_path = output_dir[:-1] + '/checkpoint-539"'
print(eval_model_name_or_path+'\n')

save_dir = output_dir[:-1] + '/'+eval_model_name_or_path[1:-1].replace('/', '?')+\
    '_bull_para_embed_merged_overlaps_bybook_gps_val.txt"'
score_dir = output_dir[:-1] + '/'+eval_model_name_or_path[1:-1].replace('/', '?')+\
    '_bull_para_embed_merged_overlaps_bybook_gps_val.json"'

!python3 $eval_script \
$eval_model_name_or_path \
$source_val_dir \
$save_dir \
--reference_path $reference_val_dir \
--score_path $score_dir \
--bs 8 \
--dump-args \
--min_length $min_length \
--length_penalty $length_penalty \
--num_beams $num_beams \
--no_repeat_ngram_size $no_repeat_ngram_size \
--max_length $config.ONE_BULLET_MAX_LEN

"/content/drive/My Drive/MAGMA: Summarization/fine-tuning/ft_pegasus_para_wordembed_merge_bybook_gas64_lr5e-05/checkpoint-539"

parsed the following generate kwargs: {'min_length': 10, 'length_penalty': 1, 'num_beams': 6, 'no_repeat_ngram_size': 3, 'max_length': 150}
  0% 0/19 [00:00<?, ?it/s]2021-03-17 15:29:55.353835: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
100% 19/19 [01:48<00:00,  5.69s/it]
{'rouge1_precision': 33.11, 'rouge1_recall': 42.68, 'rouge1_fmeasure': 34.260000000000005, 'rouge2_precision': 15.110000000000001, 'rouge2_recall': 19.09, 'rouge2_fmeasure': 15.559999999999999, 'rougeL_precision': 25.590000000000003, 'rougeL_recall': 33.31, 'rougeL_fmeasure': 26.669999999999998, 'rougeLsum_precision': 27.76, 'rougeLsum_recall': 35.86, 'rougeLsum_fmeasure': 28.82, 'sentence_distilroberta_cosine': 65.17473459243774, 'w2v_cosine': 99.99727606773376, 'n_obs': 149, 'runtime': 108, 'seconds_per_sample': 0.72

In [20]:
eval_model_name_or_path = output_dir[:-1] + '/checkpoint-539"'
print(eval_model_name_or_path+'\n')

save_dir = output_dir[:-1] + '/'+eval_model_name_or_path[1:-1].replace('/', '?')+\
    '_bull_para_embed_merged_overlaps_bybook_gps_test.txt"'
score_dir = output_dir[:-1] + '/'+eval_model_name_or_path[1:-1].replace('/', '?')+\
    '_bull_para_embed_merged_overlaps_bybook_gps_test.json"'

!python3 $eval_script \
$eval_model_name_or_path \
$source_test_dir \
$save_dir \
--reference_path $reference_test_dir \
--score_path $score_dir \
--bs 8 \
--dump-args \
--min_length $min_length \
--length_penalty $length_penalty \
--num_beams $num_beams \
--no_repeat_ngram_size $no_repeat_ngram_size \
--max_length $config.ONE_BULLET_MAX_LEN

"/content/drive/My Drive/MAGMA: Summarization/fine-tuning/ft_pegasus_para_wordembed_merge_bybook_gas64_lr5e-05/checkpoint-539"

parsed the following generate kwargs: {'min_length': 10, 'length_penalty': 1, 'num_beams': 6, 'no_repeat_ngram_size': 3, 'max_length': 150}
  0% 0/28 [00:00<?, ?it/s]2021-03-17 15:22:59.540831: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
100% 28/28 [02:23<00:00,  5.13s/it]
{'rouge1_precision': 33.12, 'rouge1_recall': 37.91, 'rouge1_fmeasure': 32.73, 'rouge2_precision': 14.219999999999999, 'rouge2_recall': 15.47, 'rouge2_fmeasure': 13.73, 'rougeL_precision': 25.44, 'rougeL_recall': 29.049999999999997, 'rougeL_fmeasure': 25.06, 'rougeLsum_precision': 27.67, 'rougeLsum_recall': 31.380000000000003, 'rougeLsum_fmeasure': 27.18, 'sentence_distilroberta_cosine': 61.9690477848053, 'w2v_cosine': 99.99718070030212, 'n_obs': 222, 'runtime': 143, 'seconds_per_sample': 0.6441, 'min_length': 10, 'leng